# Face Morphing with StyleGAN

In [0]:
import os
import pickle
import numpy as np

# Make sure tensorflow 1.15 is used
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

## 1) Encoding images into StyleGAN's latent space

#### Clone github repo for face morphing

In [0]:
# Clone github repo for face morphing
!rm -rf sample_data
!rm -rf FaceMorphing
!git clone https://github.com/antoinedandi/FaceMorphing

#### Setting up folder structure

In [0]:
# Setting up folder structure
%cd FaceMorphing
!rm -rf raw_images aligned_images generated_images guessed_images
!mkdir raw_images aligned_images generated_images guessed_images
!ls

In [0]:
# Importing useful functions / files
import dnnlib
import dnnlib.tflib as tflib
from utils.utils import display_folder_content, generate_faces_from_latent, display_encoding_results, display_results_face_morphing, display_results_face_recognition

#### **`/!\ TODO :`** *Upload Images in the raw_images folder*

#### Check uploaded faces

In [0]:
# Check uploaded faces
display_folder_content('raw_images')

#### Align Images

In [0]:
# Look for, crop and align faces. Save the resulting images in the 'aligned_images' folder
!python align_images.py raw_images/ aligned_images/ --output_size=1024

#### Check aligned images

In [0]:
# check aligned images
display_folder_content('aligned_images')
print("aligned_images contains %d images ready for encoding!" %len(os.listdir('aligned_images/')))

#### Actual encoding process

In [0]:
# TODO : Setting up options for encoding process

# General options
bs = 2             # Batch size
lr = 0.02          # Learning rate for perceptual model
it = 100           # Number of optimization steps
use_resnet = True  # Use pretrained ResNet for approximating dlatents


# Loss options
vgg_loss   = 0.4   # (default=0.4) - Use VGG perceptual loss
pixel_loss = 1.5   # (default=1.5) - Use logcosh image pixel loss
mssim_loss = 100   # (default=100) - Use MS-SIM perceptual loss
lpips_loss = 100   # (default=100) - Use LPIPS perceptual loss
l1_penalty = 1     # (default=1  ) - Use L1 penalty on latents


In [0]:
# Encoding images in latent space
!python encode_images.py --batch_size={bs} --lr={lr} --iterations={it} --use_resnet={use_resnet} --use_vgg_loss={vgg_loss} --use_pixel_loss={pixel_loss} --use_mssim_loss={mssim_loss} --use_lpips_loss={lpips_loss} --use_l1_penalty={l1_penalty} aligned_images/ generated_images/encoding/ guessed_images/encoding/ latent_representations/encoding/
print("\n************ Latent code optimization finished! ***************")

#### Check encoding results

In [0]:
# Display encoding results
display_encoding_results('aligned_images', 'guessed_images/encoding', 'generated_images/encoding')

## 2) Face Morphing by interpolating the latent space of StyleGAN

In [0]:
# TODO : Setting up options for encoding process

# General options
bs = 2             # Batch size
lr = 0.02          # Learning rate for perceptual model
it = 100           # Number of optimization steps
use_resnet = True  # Use pretrained ResNet for approximating dlatents


# Loss options
vgg_loss   = 0.4   # (default=0.4) - Use VGG perceptual loss
pixel_loss = 1.5   # (default=1.5) - Use logcosh image pixel loss
mssim_loss = 100   # (default=100) - Use MS-SIM perceptual loss
lpips_loss = 100   # (default=100) - Use LPIPS perceptual loss
l1_penalty = 1     # (default=1  ) - Use L1 penalty on latents


In [0]:
# # Perform face morphing by interpolating the latent space of StyleGAN
!python face_morphing_latent_interpolation.py --batch_size={bs} --lr={lr} --iterations={it} --use_resnet={use_resnet} --use_vgg_loss={vgg_loss} --use_pixel_loss={pixel_loss} --use_mssim_loss={mssim_loss} --use_lpips_loss={lpips_loss} --use_l1_penalty={l1_penalty} aligned_images/ generated_images/latent_interpolation/ guessed_images/latent_interpolation/ latent_representations/latent_interpolation/
print("\n************ Latent code optimization finished! ***************")

In [0]:
display_results_face_morphing('aligned_images/', 'generated_images/latent_interpolation/', res=256, fs=25)

In [0]:
display_results_face_recognition('aligned_images/', 'generated_images/latent_interpolation/', tolerance=0.6, res=256, fs=25)

## 3) Face Morphing by concurrent optimization

In [0]:
# TODO : Setting up options for encoding process

# General options
bs = 2             # Batch size
lr = 0.02          # Learning rate for perceptual model
it = 100           # Number of optimization steps
use_resnet = True  # Use pretrained ResNet for approximating dlatents
face_mask = True   # Generate a mask for predicting only the face area


# Loss options
vgg_loss   = 0.4   # (default=0.4) - Use VGG perceptual loss
pixel_loss = 1.5   # (default=1.5) - Use logcosh image pixel loss
mssim_loss = 100   # (default=100) - Use MS-SIM perceptual loss
lpips_loss = 100   # (default=100) - Use LPIPS perceptual loss
l1_penalty = 4     # (default=1  ) - Use L1 penalty on latents


In [0]:
!python face_morphing_concurrent_optimization.py --batch_size={bs} --lr={lr} --iterations={it} --use_resnet={use_resnet} --face_mask={face_mask} --use_vgg_loss={vgg_loss} --use_pixel_loss={pixel_loss} --use_mssim_loss={mssim_loss} --use_lpips_loss={lpips_loss} --use_l1_penalty={l1_penalty} aligned_images/ generated_images/concurrent_optimization/ guessed_images/concurrent_optimization/ latent_representations/concurrent_optimization/
print("\n************ Latent code optimization finished! ***************")

In [0]:
display_results_face_morphing('aligned_images/', 'generated_images/concurrent_optimization_2/', res=256, fs=25)

In [0]:
display_results_face_recognition('aligned_images/', 'generated_images/concurrent_optimization_2/', tolerance=0.6, res=256, fs=25)

In [0]:
%cd FaceMorphing
!ls

In [0]:
# Zipping results to save them

#!zip -r aligned_images.zip aligned_images/
#!zip -r guessed_images.zip guessed_images/
#!zip -r generated_images.zip generated_images/
#!zip -r latent_representations.zip latent_representations/